In [32]:
import pandas as pd
import math
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import urllib

In [2]:
level_col = 0
item_col = 1
desc_col = 2
unit_col = 6
total_col = 7

In [33]:
params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER=WINDOWS-UJU3M60\SQLEXPRESS;DATABASE=SMEC;UID=SMEC;PWD=smecs')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [22]:

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=WINDOWS-UJU3M60\SQLEXPRESS;DATABASE=SMEC;UID=SMEC;PWD=smecs')
cursor = cnxn.cursor()

cursor.execute("SELECT * FROM Form_IDs")
for row in cursor.fetchall():
    print (row)

In [3]:
#Read in Excel File to get sheet names
#excel = pd.ExcelFile("Tender Schedule (NEW Branding).xlsm")
excel = pd.ExcelFile("Tender_Schedule_test.xlsm")
sheets = excel.sheet_names

In [4]:
df_forms = pd.DataFrame(columns = {"Form ID", "Form Name"})

In [5]:
#Read pages and convert to Data Frames
#Data Frames stored in DF_List
df_list = list()
inc = 0
for x in sheets :
    df_list.append(pd.read_excel("Tender_Schedule_test.xlsm", sheet_name=x))
    form = [[inc,x]]
    df_forms = df_forms.append(pd.DataFrame(form,columns = ["Form ID", "Form Name"]),ignore_index = True)
    inc += 1

In [6]:
print(df_forms)

                          Form Name Form ID
0                   Drawings & Data       0
1                           Summary       1
2                   Road & Drainage       2
3                Sewer Retic (MRWA)       3
4                  Sewer Retic (WW)       4
5    Branch Sewer MRWA Reimbursable       5
6   Branch Sewer MRWA Reimburs (WW)       6
7                          DW Retic       7
8                     DW Retic (WW)       8
9                         NDW Retic       9
10                   NDW Retic (WW)      10
11     Trunk DW Main - Reimbursable      11
12               Trunk DW Main (WW)      12
13    Trunk NDW Main - Reimbursable      13
14            Trunk NDW Main - (WW)      14
15                       Electrical      15


In [7]:
df_col = ["Form ID", "Level 1", "Level 2", "Level 3", "Level 4", "Level 5", "Item ID","Description","Units"]
df = pd.DataFrame(columns = df_col)
print(df)

Empty DataFrame
Columns: [Form ID, Level 1, Level 2, Level 3, Level 4, Level 5, Item ID, Description, Units]
Index: []


In [8]:
for x in range(0,len(sheets)-1):
    inside = False
    for index, rows in df_list[x].iterrows():
        if inside :
            #Check if at end of table
            if rows[total_col] == "Total" :
                inside = False
            else :
                hold = ""
                df_hold = ["","0","0","0","0","0","","",""]
                full_count = 0
                rows[item_col] = str(rows[item_col])
                if rows[item_col] != "nan":
                    count = 0
                    length = len(rows[item_col])
                    for i in str(rows[item_col]):
                        count += 1
                        if i != ".":
                            hold = hold + i
                            if count == length:
                                df_hold[full_count + 1] = hold
                        else:
                            df_hold[full_count + 1] = hold
                            full_count += 1
                            hold = ""
                    df_hold[7] = rows[desc_col]
                    df_hold[0] = df_forms['Form ID'][x]
                    for y in range(0,6):
                        df_hold[6] = str(df_hold[6]) + str(df_hold[y]) + "."
                    df_hold[6] = df_hold[6][:-1]
                    df_hold[8] = rows[unit_col]
                    #PUSH TO SQL HERE
                    s_row = pd.Series(df_hold, index=df.columns)
                    df = df.append(s_row,ignore_index=True)
                    print(df_hold)       

        if rows[item_col] == "Item" :
            inside = True

[2, '1', '0', '0', '0', '0', '2.1.0.0.0.0', 'Siteworks and Earthworks', nan]
[2, '1', '1', '0', '0', '0', '2.1.1.0.0.0', 'Siteworks', nan]
[2, '1', '1', '1', '0', '0', '2.1.1.1.0.0', 'Site establishment including provision of access, survey setout, provision of site and office compounds, insurances and other works as specified.', 'Item']
[2, '1', '1', '2', '0', '0', '2.1.1.2.0.0', 'Site preparation including disposal of all surface rock, trees and stumps, timber, fences, wire, rubbish, removal of kerb and other existing works and any other materials as specified on the drawings, disposed offsite to a tip to be arranged by the contractor.', 'Item']
[2, '1', '2', '0', '0', '0', '2.1.2.0.0.0', 'Bulk Earthworks ', nan]
[2, '1', '2', '1', '0', '0', '2.1.2.1.0.0', 'Strip existing topsoil and stockpile for future use (Refer to geotech report for depths)', 'sq.m']
[2, '1', '2', '2', '0', '0', '2.1.2.2.0.0', 'Cut material', 'cu.m']
[2, '1', '2', '3', '0', '0', '2.1.2.3.0.0', 'Spread and compact

[2, '3', '1', '3', '0', '0', '2.3.1.3.0.0', 'Rollover kerb and channel (outfall tray)', 'lin.m.']
[2, '3', '1', '4', '0', '0', '2.3.1.4.0.0', 'Semi mountable kerb (SM1)', 'lin.m.']
[2, '3', '1', '5', '0', '0', '2.3.1.5.0.0', 'Semi mountable kerb and channel (SM2)', 'lin.m.']
[2, '3', '1', '6', '0', '0', '2.3.1.6.0.0', 'Semi mountable kerb and channel (SM3)', 'lin.m.']
[2, '3', '1', '7', '0', '0', '2.3.1.7.0.0', 'Barrier kerb (B1)', 'lin.m.']
[2, '3', '1', '8', '0', '0', '2.3.1.8.0.0', 'Barrier kerb and channel (B2) -  600mm', 'lin.m.']
[2, '3', '1', '9', '0', '0', '2.3.1.9.0.0', 'Barrier kerb and channel (B3) - 450mm / 600mm', 'lin.m.']
[2, '3', '1', '10', '0', '0', '2.3.1.10.0.0', 'Mountable kerb and channel (M2)', 'lin.m.']
[2, '3', '1', '11', '0', '0', '2.3.1.11.0.0', 'Edge strip', 'lin.m.']
[2, '3', '2', '0', '0', '0', '2.3.2.0.0.0', '100mm diameter subsoil drain (including localised deepening of capping layer)', 'lin.m.']
[2, '3', '2', '1', '0', '0', '2.3.2.1.0.0', '100mm diameter

[2, '6', '1', '3', '0', '0', '2.6.1.3.0.0', 'Relocate existing edge strip, "NO ROAD" sign & barrier', 'no.']
[2, '6', '2', '0', '0', '0', '2.6.2.0.0.0', 'Signs', nan]
[2, '6', '3', '0', '0', '0', '2.6.3.0.0.0', 'PSM', 'no.']
[2, '6', '4', '0', '0', '0', '2.6.4.0.0.0', 'Retaining wall (includes building permits)', 'lin.m']
[2, '6', '4', '1', '0', '0', '2.6.4.1.0.0', 'Stone retaining wall', nan]
[2, '6', '4', '2', '0', '0', '2.6.4.2.0.0', 'Concrete retaining wall', nan]
[2, '6', '4', '3', '0', '0', '2.6.4.3.0.0', 'Timber retaining wall', nan]
[2, '6', '5', '0', '0', '0', '2.6.5.0.0.0', 'Standard paling fence (Specify Height 1.5m, 1.8m, 2.1m)', 'lin.m']
[2, '6', '6', '0', '0', '0', '2.6.6.0.0.0', 'Bollards (SPECIFY)', 'no.']
[2, '6', '7', '0', '0', '0', '2.6.7.0.0.0', 'Linemarking', 'Item']
[2, '6', '8', '0', '0', '0', '2.6.8.0.0.0', "RRPM's", 'no.']
[2, '6', '9', '0', '0', '0', '2.6.9.0.0.0', 'Guard rail (SPECIFY)', 'lin.m']
[2, '6', '10', '0', '0', '0', '2.6.10.0.0.0', 'Wire rope (SPECI

[3, '1', '2', '3', '2', '0', '3.1.2.3.2.0', '1.5-2.5m deep', 'lin.m']
[3, '1', '2', '3', '3', '0', '3.1.2.3.3.0', '2.5-3.5m deep', 'lin.m']
[3, '1', '2', '4', '0', '0', '3.1.2.4.0.0', '300mm dia.', nan]
[3, '1', '2', '4', '1', '0', '3.1.2.4.1.0', '<1.5m deep', 'lin.m']
[3, '1', '2', '4', '2', '0', '3.1.2.4.2.0', '1.5-2.5m deep', 'lin.m']
[3, '1', '2', '4', '3', '0', '3.1.2.4.3.0', '2.5-3.5m deep', 'lin.m']
[3, '1', '3', '0', '0', '0', '3.1.3.0.0.0', 'Extra over item 1.1 for Type R backfill (MRWA-S-201)', nan]
[3, '1', '3', '1', '0', '0', '3.1.3.1.0.0', '100mm dia.', nan]
[3, '1', '3', '1', '1', '0', '3.1.3.1.1.0', '<1.5m deep', 'lin.m']
[3, '1', '3', '1', '2', '0', '3.1.3.1.2.0', '1.5-2.5m deep', 'lin.m']
[3, '1', '3', '1', '3', '0', '3.1.3.1.3.0', '2.5-3.5m deep', 'lin.m']
[3, '1', '3', '2', '0', '0', '3.1.3.2.0.0', '150mm dia.', nan]
[3, '1', '3', '2', '1', '0', '3.1.3.2.1.0', '<1.5m deep', 'lin.m']
[3, '1', '3', '2', '2', '0', '3.1.3.2.2.0', '1.5-2.5m deep', 'lin.m']
[3, '1', '3', '

[3, '7', '1', '6', '0', '0', '3.7.1.6.0.0', 'Type 4S shaft connections', 'no.']
[3, '7', '1', '7', '0', '0', '3.7.1.7.0.0', 'Type 4B base connections', 'no.']
[3, '7', '1', '8', '0', '0', '3.7.1.8.0.0', 'Type S shaft connections', 'no.']
[3, '7', '1', '9', '0', '0', '3.7.1.9.0.0', 'Type B base connections', 'no.']
[3, '7', '1', '10', '0', '0', '3.7.1.10.0.0', 'Type OB', 'no.']
[3, '7', '2', '0', '0', '0', '3.7.2.0.0.0', '150mm dia. including CRB in accordance with MRWA 04-03.1 (SEW &YVW)/ MRWA 04-03.2 (CWW) (Refer MRWA-S-301 to 304)', nan]
[3, '7', '2', '1', '0', '0', '3.7.2.1.0.0', 'Type 1a', 'no.']
[3, '7', '2', '2', '0', '0', '3.7.2.2.0.0', 'Type 1b', 'no.']
[3, '7', '2', '3', '0', '0', '3.7.2.3.0.0', 'Type 2 ', 'no.']
[3, '7', '2', '4', '0', '0', '3.7.2.4.0.0', 'Type 4a', 'no.']
[3, '7', '2', '5', '0', '0', '3.7.2.5.0.0', 'Type 4b', 'no.']
[3, '7', '2', '6', '0', '0', '3.7.2.6.0.0', 'Type 4S shaft connections', 'no.']
[3, '7', '2', '7', '0', '0', '3.7.2.7.0.0', 'Type 4B base connec

...

[8, '2', '6', '4', '4', '0', '8.2.6.4.4.0', '300mm', 'no.']
[8, '2', '7', '0', '0', '0', '8.2.7.0.0.0', 'Puddle Flange (Refer MRWA-W-205A)', nan]
[8, '2', '7', '1', '0', '0', '8.2.7.1.0.0', '100mm', 'no.']
[8, '2', '7', '2', '0', '0', '8.2.7.2.0.0', '150mm', 'no.']
[8, '2', '7', '3', '0', '0', '8.2.7.3.0.0', '225mm', 'no.']
[8, '2', '7', '4', '0', '0', '8.2.7.4.0.0', '300mm', 'no.']
[8, '2', '8', '0', '0', '0', '8.2.8.0.0.0', 'Reducer (Refer MRWA-W-205A)', nan]
[8, '2', '8', '1', '0', '0', '8.2.8.1.0.0', '150mm to 100mm', 'no.']
[8, '2', '8', '2', '0', '0', '8.2.8.2.0.0', '225mm to 150mm', 'no.']
[8, '2', '8', '3', '0', '0', '8.2.8.3.0.0', '300mm to 225mm', 'no.']
[8, '3', '0', '0', '0', '0', '8.3.0.0.0.0', 'Thrust blocks', nan]
[8, '3', '1', '0', '0', '0', '8.3.1.0.0.0', 'Concrete (Dual Main or Solo Main Requirement) (Refer MRWA-W-205A & 205B)', nan]
[8, '3', '1', '1', '0', '0', '8.3.1.1.0.0', 'A - TYPE - SIZE', 'no.']
[8, '3', '1', '2', '0', '0', '8.3.1.2.0.0', 'B - TYPE - SIZE', 'no

[9, '2', '6', '4', '1', '0', '9.2.6.4.1.0', '100mm', nan]
[9, '2', '6', '4', '2', '0', '9.2.6.4.2.0', '150mm', 'no.']
[9, '2', '6', '4', '3', '0', '9.2.6.4.3.0', '225mm', 'no.']
[9, '2', '6', '4', '4', '0', '9.2.6.4.4.0', '300mm', 'no.']
[9, '2', '7', '0', '0', '0', '9.2.7.0.0.0', 'Puddle Flange (Refer MRWA-W-205A)', nan]
[9, '2', '7', '1', '0', '0', '9.2.7.1.0.0', '100mm', 'no.']
[9, '2', '7', '2', '0', '0', '9.2.7.2.0.0', '150mm', 'no.']
[9, '2', '7', '3', '0', '0', '9.2.7.3.0.0', '225mm', 'no.']
[9, '2', '7', '4', '0', '0', '9.2.7.4.0.0', '300mm', 'no.']
[9, '2', '8', '0', '0', '0', '9.2.8.0.0.0', 'Reducer (Refer MRWA-W-205A)', nan]
[9, '2', '8', '1', '0', '0', '9.2.8.1.0.0', '150mm to 100mm', 'no.']
[9, '2', '8', '2', '0', '0', '9.2.8.2.0.0', '225mm to 150mm', 'no.']
[9, '2', '8', '3', '0', '0', '9.2.8.3.0.0', '300mm to 225mm', 'no.']
[9, '3', '0', '0', '0', '0', '9.3.0.0.0.0', 'Thrust blocks \nAll dual main thrust blocks are quantified and tendered as part of DW costings', nan]
[9

[10, '2', '6', '1', '4', '0', '10.2.6.1.4.0', '300mm', 'no.']
[10, '2', '6', '2', '0', '0', '10.2.6.2.0.0', 'Bends 22.5° x XXXmm (Refer MRWA-W-205A)', 'no.']
[10, '2', '6', '2', '1', '0', '10.2.6.2.1.0', '100mm', nan]
[10, '2', '6', '2', '2', '0', '10.2.6.2.2.0', '150mm', 'no.']
[10, '2', '6', '2', '3', '0', '10.2.6.2.3.0', '225mm', 'no.']
[10, '2', '6', '2', '4', '0', '10.2.6.2.4.0', '300mm', 'no.']
[10, '2', '6', '3', '0', '0', '10.2.6.3.0.0', 'Bends 45° x XXXmm (Refer MRWA-W-205A)', 'no.']
[10, '2', '6', '3', '1', '0', '10.2.6.3.1.0', '100mm', nan]
[10, '2', '6', '3', '2', '0', '10.2.6.3.2.0', '150mm', 'no.']
[10, '2', '6', '3', '3', '0', '10.2.6.3.3.0', '225mm', 'no.']
[10, '2', '6', '3', '4', '0', '10.2.6.3.4.0', '300mm', 'no.']
[10, '2', '6', '4', '0', '0', '10.2.6.4.0.0', 'Bends 90° x XXXmm (Refer MRWA-W-205A)', 'no.']
[10, '2', '6', '4', '1', '0', '10.2.6.4.1.0', '100mm', nan]
[10, '2', '6', '4', '2', '0', '10.2.6.4.2.0', '150mm', 'no.']
[10, '2', '6', '4', '3', '0', '10.2.6.4.

[12, '3', '5', '0', '0', '0', '12.3.5.0.0.0', 'Chlorination assembly (Refer MWRA-W-308)', nan]
[12, '3', '5', '1', '0', '0', '12.3.5.1.0.0', 'At 300mm dia main', 'no.']
[12, '3', '5', '2', '0', '0', '12.3.5.2.0.0', 'At 375mm dia main', 'no.']
[12, '3', '5', '3', '0', '0', '12.3.5.3.0.0', 'At 450mm dia main', 'no.']
[12, '3', '6', '0', '0', '0', '12.3.6.0.0.0', 'Cross connections (including all pipework and valve fittings as specified) ', 'no.']
[12, '4', '0', '0', '0', '0', '12.4.0.0.0.0', 'Thrust blocks', nan]
[12, '4', '1', '0', '0', '0', '12.4.1.0.0.0', 'Concrete (Dual Main or Solo Main Requirement) (Refer MRWA-W-205A & 205B)', nan]
[12, '4', '1', '1', '0', '0', '12.4.1.1.0.0', 'A - TYPE - SIZE', 'no.']
[12, '4', '1', '2', '0', '0', '12.4.1.2.0.0', 'B - TYPE - SIZE', 'no.']
[12, '4', '1', '3', '0', '0', '12.4.1.3.0.0', 'C - TYPE - SIZE', 'no.']
[12, '4', '2', '0', '0', '0', '12.4.2.0.0.0', 'Timber and recycled plastic (Dual Main or Solo Main Requirement) (Refer MRWA-W-206)', 'no.']


[13, '5', '1', '0', '0', '0', '13.5.1.0.0.0', 'Remove fitting and connect to existing XXX dia. Main (Refer MRWA-W-106)', 'no.']
[13, '5', '2', '0', '0', '0', '13.5.2.0.0.0', 'Cut, seal and abandon existing watermain', 'Item']
[13, '6', '0', '0', '0', '0', '13.6.0.0.0.0', 'Miscellaneous', nan]
[13, '6', '1', '0', '0', '0', '13.6.1.0.0.0', 'Cathodic protection for MSCL mains as per specification', 'Item']
[13, '7', '0', '0', '0', '0', '13.7.0.0.0.0', 'Swabbing, testing and cleansing of mains (Items include supply of all material)', nan]
[13, '7', '1', '0', '0', '0', '13.7.1.0.0.0', 'Swabbing and cleansing of mains', 'Item']
[13, '7', '2', '0', '0', '0', '13.7.2.0.0.0', 'Pressure testing of mains', 'Item']
[13, '8', '0', '0', '0', '0', '13.8.0.0.0.0', 'Quality Assurance', nan]
[13, '8', '1', '0', '0', '0', '13.8.1.0.0.0', 'Quality testing of works including certification at start and end of DLP. Inclusive of testing to meet requirements of MRWA 04-03.2 Backfill Specification.', 'Item']
[1

In [24]:
df = df.reset_index(drop=True)

In [36]:
df.to_sql('Tender Master', engine, if_exists='replace')

In [37]:
df_forms.to_sql('Form_IDs', engine, if_exists='replace')

In [9]:
df.to_csv("test.csv")

In [10]:
df_forms.to_csv("Form_IDs.csv")